In [0]:
# for some basic operations
import numpy as np 
import pandas as pd 

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
base_dp = 'https://drive.google.com/open?id=1kf85Br9YVKUCTAFt1XQAWnVgXkyh2qsx'

fluff, id_2 = base_dp.split('=')

print (id) # Verify that you have everything after '='
 
downloaded = drive.CreateFile({'id':id_2}) 
downloaded.GetContentFile('BaseDPEvolucaoMensalCisp.csv')


<built-in function id>


In [41]:
crimes_df = pd.read_csv('BaseDPEvolucaoMensalCisp.csv', encoding='latin-1')

crimes_df = crimes_df.replace(r'^\s*$', np.nan, regex=True)  #campos com apenas espaços em branco são substituidos por NaN
crimes_df = crimes_df.fillna(0) # substitui todos os campos vazios com 0

crimes_df['total_roubos'] = crimes_df['total_roubos'] - crimes_df['latrocinio']
crimes_df['outros_roubos'] = crimes_df['outros_roubos'] - crimes_df['latrocinio']

crimes_df['munic'] = crimes_df['munic'].str.strip()
crimes_df['Regiao'] = crimes_df['Regiao'].str.strip()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (44,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Eliminando colunas inúteis**

In [42]:
crimes_df.drop(['CISP', 'mes_ano', 'mes', 'vano', 'AISP','RISP', 'mcirc', 'munic', 'Regiao', 'fase','total_roubos', 'total_furtos','indicador_letalidade',
         'indicador_roubo_rua', 'indicador_roubo_veic', 'registro_ocorrencias', 'posse_drogas', 'trafico_drogas', 'apreensao_drogas_sem_autor', 
         'outros_roubos', 'outros_furtos', 'apf', 'aaapai', 'cmp', 'cmba', 'encontro_cadaver', 'encontro_ossada' ], axis=1, inplace =True)
crimes_df.head()

,hom_doloso,lesao_corp_morte,latrocinio,hom_por_interv_policial,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,lesao_corp_culposa,roubo_comercio,roubo_residencia,roubo_veiculo,roubo_carga,roubo_transeunte,roubo_em_coletivo,roubo_banco,roubo_cx_eletronico,roubo_celular,roubo_conducao_saque,roubo_apos_saque,roubo_bicicleta,furto_veiculos,furto_transeunte,furto_coletivo,furto_celular,furto_bicicleta,sequestro,extorsao,sequestro_relampago,estelionato,apreensao_drogas,recuperacao_veiculos,ameaca,pessoas_desaparecidas,pol_militares_mortos_serv,pol_civis_mortos_serv
0,0,0,0,0,1,40,0,1,15,14,0,5,1,26,8,0,0.0,32,0,10,0.0,12,30,0,37,0.0,0,1,0,69,1,5,21,2,0,0
1,3,0,0,0,0,47,1,4,35,5,0,9,1,25,12,0,1.0,14,1,3,0.0,9,42,5,23,0.0,0,1,0,1,35,7,15,6,0,0
2,3,0,0,0,1,73,2,1,19,10,1,11,5,26,4,2,0.0,34,2,4,0.0,28,42,2,47,0.0,0,0,0,37,4,10,47,2,0,0
3,6,0,0,0,2,43,2,1,20,10,0,27,6,14,22,0,0.0,20,0,6,0.0,17,4,0,8,0.0,0,0,0,8,20,77,26,2,0,0
4,4,0,0,0,2,18,2,0,2,0,2,23,1,4,0,0,0.0,1,0,1,0.0,12,1,0,1,0.0,0,0,0,4,3,9,10,1,0,0


In [43]:
media = crimes_df.mean()
media

hom_doloso                    3.314665
lesao_corp_morte              0.027760
latrocinio                    0.112572
hom_por_interv_policial       0.602690
tentat_hom                    3.017000
lesao_corp_dolosa            47.762563
estupro                       2.764394
hom_culposo                   1.453839
lesao_corp_culposa           24.340856
roubo_comercio                3.656305
roubo_residencia              0.930469
roubo_veiculo                20.402578
roubo_carga                   3.332636
roubo_transeunte             38.079918
roubo_em_coletivo             5.900691
roubo_banco                   0.018943
roubo_cx_eletronico           0.026527
roubo_celular                 7.268933
roubo_conducao_saque          0.084588
roubo_apos_saque              1.088474
roubo_bicicleta               0.027461
furto_veiculos               11.543583
furto_transeunte             15.763572
furto_coletivo                4.858808
furto_celular                10.828881
furto_bicicleta          

Transformando valores em booleanos: acima da média é True, abaixo da média é False

In [45]:
i = 0
for c in crimes_df.columns:
  crimes_df[c] = crimes_df[c] >= media[i]
  i += 1
  
crimes_df.head()

,hom_doloso,lesao_corp_morte,latrocinio,hom_por_interv_policial,tentat_hom,lesao_corp_dolosa,estupro,hom_culposo,lesao_corp_culposa,roubo_comercio,roubo_residencia,roubo_veiculo,roubo_carga,roubo_transeunte,roubo_em_coletivo,roubo_banco,roubo_cx_eletronico,roubo_celular,roubo_conducao_saque,roubo_apos_saque,roubo_bicicleta,furto_veiculos,furto_transeunte,furto_coletivo,furto_celular,furto_bicicleta,sequestro,extorsao,sequestro_relampago,estelionato,apreensao_drogas,recuperacao_veiculos,ameaca,pessoas_desaparecidas,pol_militares_mortos_serv,pol_civis_mortos_serv
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [46]:
crimes_df.shape

(26765, 36)

### Apriori 

Apriori é um algoritmo popular para extrair itemsets frequentes com aplicações no aprendizado de regras de associação. O algoritmo apriori foi projetado para operar em bancos de dados contendo transações, como compras de clientes de uma loja. Um itemset é considerado "frequente" se atingir um limite de suporte especificado pelo usuário. Por exemplo, se o limite de suporte for definido como 0,5 (50%), um itemset frequente será definido como um conjunto de itens que ocorrem juntos em pelo menos 50% de todas as transações no banco de dados.

Um conceito-chave no algoritmo Apriori é a anti-monotonicidade da medida de suporte. Pressupõe que:

* Todos os subconjuntos de um itemset frequente devem ser frequentes
* Da mesma forma, para qualquer itemset pouco frequente, todos os seus superconjuntos também devem ser pouco frequentes

O algoritmo APRIORI é dividido em duas partes. Na primeira parte são selecionados todos os subconjuntos de T que podem ser utilizados em alguma regra, ou seja, que contenham o suporte acima do suporte mínimo S. A segunda parte do algoritmo faz a geração das regras a partir dos subconjuntos gerados na primeira parte, sendo que estas regras devem ter uma confiança maior que a confiança mínima C.

In [51]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#Now, let us return the items and itemsets with at least 1% support:
frequent_itemsets = apriori(crimes_df, min_support = 0.01, use_colnames = True)

frequent_itemsets.head()


,support,itemsets
0,0.025593,(lesao_corp_morte)
1,0.090192,(latrocinio)
2,0.238595,(hom_por_interv_policial)
3,0.472670,(roubo_residencia)
4,0.018046,(roubo_banco)


In [52]:
frequent_itemsets.shape

(58, 2)

In [53]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(hom_por_interv_policial),(lesao_corp_morte),0.238595,0.025593,0.011508,0.048231,1.884510,0.005401,1.023785
1,(lesao_corp_morte),(hom_por_interv_policial),0.025593,0.238595,0.011508,0.449635,1.884510,0.005401,1.383454
2,(roubo_residencia),(lesao_corp_morte),0.472670,0.025593,0.015879,0.033594,1.312625,0.003782,1.008279
3,(lesao_corp_morte),(roubo_residencia),0.025593,0.472670,0.015879,0.620438,1.312625,0.003782,1.389313
4,(extorsao),(lesao_corp_morte),0.450476,0.025593,0.015206,0.033756,1.318961,0.003677,1.008448
5,(lesao_corp_morte),(extorsao),0.025593,0.450476,0.015206,0.594161,1.318961,0.003677,1.354042
6,(hom_por_interv_policial),(latrocinio),0.238595,0.090192,0.041584,0.174288,1.932396,0.020065,1.101845
7,(latrocinio),(hom_por_interv_policial),0.090192,0.238595,0.041584,0.461060,1.932396,0.020065,1.412783
8,(roubo_residencia),(latrocinio),0.472670,0.090192,0.061424,0.129950,1.440811,0.018792,1.045696
9,(latrocinio),(roubo_residencia),0.090192,0.472670,0.061424,0.681027,1.440811,0.018792,1.653215


In [54]:
rules.shape

(188, 9)

Queremos apenas ver as regras em que a confiança é maior ou igual a 70%, portanto:

In [60]:
rules[(rules['lift'] >= 1) & (rules['confidence'] >= 0.7)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
44,(roubo_cx_eletronico),(extorsao),0.024622,0.450476,0.018420,0.748103,1.660694,0.007328,2.181543
47,(roubo_conducao_saque),(extorsao),0.068896,0.450476,0.051298,0.744577,1.652866,0.020262,2.151426
48,(roubo_bicicleta),(furto_bicicleta),0.021147,0.122660,0.016253,0.768551,6.265694,0.013659,3.790644
50,(roubo_bicicleta),(extorsao),0.021147,0.450476,0.015954,0.754417,1.674709,0.006427,2.237629
55,(sequestro_relampago),(extorsao),0.040052,0.450476,0.031160,0.777985,1.727027,0.013117,2.475165
57,"(hom_por_interv_policial, latrocinio)",(roubo_residencia),0.041584,0.472670,0.030786,0.740341,1.566298,0.011131,2.030861
70,"(extorsao, latrocinio)",(roubo_residencia),0.057314,0.472670,0.041920,0.731421,1.547426,0.014830,1.963410
75,"(hom_por_interv_policial, roubo_conducao_saque)",(roubo_residencia),0.029292,0.472670,0.021110,0.720663,1.524666,0.007264,1.887795
93,"(hom_por_interv_policial, sequestro_relampago)",(roubo_residencia),0.015468,0.472670,0.010947,0.707729,1.497303,0.003636,1.804255
99,"(hom_por_interv_policial, roubo_conducao_saque)",(extorsao),0.029292,0.450476,0.021745,0.742347,1.647915,0.008549,2.132805
